In [1]:
import pickle
import numpy as np 
import pandas as pd 
import math

import sys
sys.path.append('../utils')
from treasury_base import *
from scipy.stats import t
from sklearn.metrics import mean_squared_error as mse

In [6]:
cross_val_window = 20
with open(f"pickle/dfs_{cross_val_window}_steps_ahead.pkl", "rb") as file:
    loaded_dataframes = pickle.load(file)

# Extract individual DataFrames
naive_df = loaded_dataframes["naive_df"]
kan_df = loaded_dataframes["kan_df"]
truth_df = loaded_dataframes["truth_df"]

# Use the DataFrames
kan_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,4.721941,4.710064,4.620182,4.481312,4.333126,4.199277,4.175072,4.211883,4.219243,4.330094,4.609832,4.548126
1,4.684460,4.700264,4.622948,4.489969,4.345800,4.220215,4.184787,4.221259,4.229493,4.333800,4.607117,4.541424
2,4.648227,4.712086,4.628132,4.491214,4.313974,4.215311,4.174042,4.219329,4.228004,4.322638,4.606626,4.522977
3,4.648029,4.707223,4.631021,4.500008,4.318971,4.215912,4.194903,4.221367,4.223881,4.326705,4.608325,4.532009
4,4.682420,4.698236,4.634769,4.510997,4.367711,4.196893,4.206953,4.216696,4.210620,4.328431,4.590787,4.524111


In [13]:
cols = truth_df.columns
mode = 'two-sided'
cross_val_list = [1, 5, 10, 20]

result = dict()
for i in range(12):
    result[i] = list()

for cross_val_window in cross_val_list:
    with open(f"pickle/dfs_{cross_val_window}_steps_ahead.pkl", "rb") as file:
        loaded_dataframes = pickle.load(file)

    # Extract individual DataFrames
    naive_df = loaded_dataframes["naive_df"]
    kan_df = loaded_dataframes["kan_df"]
    truth_df = loaded_dataframes["truth_df"]

    for i in range(12):
        y1 = kan_df.iloc[:, i].values
        y2 = naive_df.iloc[:, i].values
        y = truth_df.iloc[:, i].values

        # print(f'{cols[i]}: {dm_test(y, y1, y2, h=1, mode=mode)}')
        if mse(y1, y) < mse(y2, y):
            result[i].append(True)
        else:
            result[i].append(False)

out = pd.DataFrame(result)
out.columns = cols
out.index = cross_val_list
out


,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
1,False,False,False,False,False,False,False,False,False,False,False,False
5,True,False,False,False,False,False,False,False,False,False,False,False
10,True,False,True,False,False,True,True,True,True,True,True,True
20,False,True,True,False,False,True,False,False,False,True,True,False


In [14]:
print(mse(kan_df.values, truth_df.values))
print(mse(naive_df.values, truth_df.values))

0.02186088156541866
0.012309166666666647


In [38]:
print(mse(kan_df.values, truth_df.values))
print(mse(naive_df.values, truth_df.values))

0.013525092183985113
0.002106250000000003
